In [8]:
import fastf1
from fastf1 import _api as api
from fastf1.logger import (
    get_logger,
    soft_exceptions
)
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import datetime

session = fastf1.get_session(2021, "Abu", 'R')

logger      WARNING 	Failed to load schedule from FastF1 backend!
req            INFO 	No cached data found for season_schedule. Loading data...
_api           INFO 	Fetching season schedule...
req            INFO 	Data has been written to cache!


In [9]:
_logger = get_logger('api')

EMPTY_LAPS = {'Time': pd.NaT, 'Driver': '', 'LapTime': pd.NaT,
              'NumberOfLaps': np.nan, 'NumberOfPitStops': np.nan,
              'PitOutTime': pd.NaT, 'PitInTime': pd.NaT,
              'Sector1Time': pd.NaT, 'Sector2Time': pd.NaT,
              'Sector3Time': pd.NaT, 'Sector1SessionTime': pd.NaT,
              'Sector2SessionTime': pd.NaT, 'Sector3SessionTime': pd.NaT,
              'SpeedI1': np.nan, 'SpeedI2': np.nan, 'SpeedFL': np.nan,
              'SpeedST': np.nan, 'IsPersonalBest': False}

EMPTY_STREAM = {'Time': pd.NaT, 'Driver': '', 'Position': np.nan,
                'GapToLeader': np.nan, 'IntervalToPositionAhead': np.nan}

In [10]:
def _extended_timing_data(path, response=None, livedata=None):
    # extended over the documentation of ``timing_data``:
    #   - returns session_split_times for splitting Q1/Q2/Q3 additionally
    # possible optional sanity checks (TODO, maybe):
    #   - inlap has to be followed by outlap
    #   - pit stops may never be negative (missing outlap)
    #   - speed traps against telemetry (especially in Q FastLap - Slow Lap)
    if livedata is not None and livedata.has('TimingData'):
        response = livedata.get('TimingData')
    elif response is None:  # no previous response provided
        _logger.info("Fetching timing data...")
        response = api.fetch_page(path, 'timing_data')
        if response is None:  # no response received
            raise SessionNotAvailableError(
                "No data for this session! If this session only finished "
                "recently, please try again in a few minutes."
            )
    _logger.info("Parsing timing data...")

    # split up response per driver for easier iteration and processing later
    resp_per_driver = dict()
    for entry in response:
        if (len(entry) < 2) or 'Lines' not in entry[1]:
            continue
        for drv in entry[1]['Lines']:
            if drv not in resp_per_driver.keys():
                resp_per_driver[drv] = [(entry[0], entry[1]['Lines'][drv])]
            else:
                resp_per_driver[drv].append((entry[0], entry[1]['Lines'][drv]))

    # create empty data dicts and populate them with data from all drivers after that
    laps_data = {key: list() for key, val in EMPTY_LAPS.items()}
    stream_data = {key: list() for key, val in EMPTY_STREAM.items()}

    session_split_times = [datetime.timedelta(days=1), ] * 3

    return resp_per_driver



resp_per_driver = _extended_timing_data(session.api_path, livedata=None)

1778511725       INFO 	Fetching timing data...
1778511725       INFO 	Parsing timing data...


In [29]:
from fastf1.utils import (
    recursive_dict_get,
    to_datetime,
    to_timedelta
)

def _laps_data_driver(driver_raw, empty_vals, drv):
    """
    .. warning::
        :mod:`fastf1.api` will be considered private in future releases and
        potentially be removed or changed.

    Data is on a per-lap basis.

    Boolean flag 'PitOut' is not evaluated. Meaning is unknown and flag is only sometimes present when a car leaves
    the pits.

    Params:
        driver_raw (list): raw api response for this driver only [(Timestamp, data), (...), ...]
        empty_vals (dict): dictionary of column names and empty column values
        drv (str): driver identifier

    Returns:
         dictionary of laps data for this driver
    """

    integrity_errors = list()

    # do a quick first pass over the data to find out when laps start and end
    # this is needed so we can work with a more efficient "look ahead" on the main pass
    # example: we can have 'PitOut' 0.01s before a new lap starts, but 'PitOut' belongs to the new lap, not the old one

    lapcnt = 0  # we're keeping two separate lap counts because sometimes the api has a non existent lap too much...
    api_lapcnt = 0  # ...at the beginning; we can correct that though;
    # api_lapcnt does not count backwards even if the source data does
    in_past = False  # flag for when the data went back in time
    out_of_pit = False  # flag set to true when driver drives out FOR THE FIRST TIME; stays true from then on

    # entries are prefilled with empty values and only overwritten if they exist in the response line
    drv_data = {key: [val, ] for key, val in empty_vals.items()}

    for time, resp in driver_raw:
        # the first three ifs are just edge case handling for the rare sessions were the data goes back in time
        if in_past and 'NumberOfLaps' in resp and resp['NumberOfLaps'] == api_lapcnt:
            in_past = False  # we're back in the present

        if 'NumberOfLaps' in resp and ((prev_lapcnt := resp['NumberOfLaps']) < api_lapcnt):
            _logger.warning(f"Driver {drv: >2}: Ignoring late data for a "
                            f"previously processed lap.The data may contain "
                            f"errors (previous: {prev_lapcnt}; "
                            f"current {lapcnt})")
            in_past = True
            continue

        if in_past:  # still in the past, just continue and ignore everything
            continue

        if ('InPit' in resp) and (resp['InPit'] is False):
            out_of_pit = True  # drove out of the pits for the first time

        # new lap; create next row
        if 'NumberOfLaps' in resp and resp['NumberOfLaps'] > api_lapcnt:
            api_lapcnt += 1
            # make sure the car actually drove out of the pits already; it can't be a new lap if it didn't
            if out_of_pit:
                drv_data['Time'][lapcnt] = to_timedelta(time)
                lapcnt += 1
                # append a new empty row; last row may not be populated (depending on session) and may be removed later
                for key, val in empty_vals.items():
                    drv_data[key].append(val)

    # now, do the main pass where all the other data is actually filled in
    # same counters and flags as before, reset them
    lapcnt = 0  # we're keeping two separate lap counts because sometimes the api has a non existent lap too much...
    api_lapcnt = 0  # ...at the beginning; we can correct that though;
    # api_lapcnt does not count backwards even if the source data does
    in_past = False  # flag for when the data went back in time

    personal_best_lap_times = list()

    session_split_times = [datetime.timedelta(0)]
    # start times of (sub)sessions (Q1, Q2, Q3)

    pitstops = -1  # start with -1 because first is out lap, needs to be zero after that

    # iterate through the data; new lap triggers next row in data
    for time, resp in driver_raw:
        # the first three ifs are just edge case handling for the rare sessions were the data goes back in time
        if in_past and 'NumberOfLaps' in resp and resp['NumberOfLaps'] == api_lapcnt:
            in_past = False  # we're back in the present
        if in_past or ('NumberOfLaps' in resp and resp['NumberOfLaps'] < api_lapcnt):
            in_past = True
            continue

        # values which are up to five seconds late are still counted towards the previous lap
        # (sector times, speed traps and lap times)
        lap_offset = 0
        if (lapcnt > 0) and (to_timedelta(time) - drv_data['Time'][lapcnt - 1] < pd.Timedelta(5, 's')):
            lap_offset = 1

        if 'Sectors' in resp and isinstance(resp['Sectors'], dict):
            # sometimes it's a list but then it never contains values...
            for sn, sector, sesst in (('0', 'Sector1Time', 'Sector1SessionTime'),
                                      ('1', 'Sector2Time', 'Sector2SessionTime'),
                                      ('2', 'Sector3Time', 'Sector3SessionTime')):
                if val := recursive_dict_get(resp, 'Sectors', sn, 'Value'):
                    drv_data[sector][lapcnt - lap_offset] = to_timedelta(val)
                    drv_data[sesst][lapcnt - lap_offset] = to_timedelta(time)
        

        if val := recursive_dict_get(resp, 'LastLapTime', 'Value'):
            # if 'LastLapTime' is received less than five seconds after the start of a new lap, it is still added
            # to the last lap
            val = to_timedelta(val)
            if val.total_seconds() < 150:
                # laps which are longer than 150 seconds are ignored; usually this is the case between Q1, Q2 and Q3
                # because all three qualifying sessions are one session here. Those timestamps are often wrong and
                # sometimes associated with the wrong lap
                drv_data['LapTime'][lapcnt - lap_offset] = val

        if 'Speeds' in resp:
            for trapkey, trapname in (('I1', 'SpeedI1'), ('I2', 'SpeedI2'), ('FL', 'SpeedFL'), ('ST', 'SpeedST')):
                if val := recursive_dict_get(resp, 'Speeds', trapkey, 'Value'):
                    # speed has to be float because int does not support NaN
                    if trapkey == 'ST':
                        # the ST trap value can occur early enough in a new lap
                        # that it needs to be excluded from the usual offset
                        # logic, therefore the offset is ignored here
                        drv_data[trapname][lapcnt] = float(val)
                    else:
                        drv_data[trapname][lapcnt - lap_offset] = float(val)

        if 'InPit' in resp:
            # 'InPit': True is received once when entering pits, False is received once when leaving
            if resp['InPit'] is True:
                if pitstops >= 0:
                    drv_data['PitInTime'][lapcnt] = to_timedelta(time)
            elif ((('NumberOfLaps' in resp) and resp['NumberOfLaps'] > api_lapcnt)
                  or (drv_data['Time'][lapcnt] - to_timedelta(time))
                  < pd.Timedelta(5, 's')):
                # same response line as beginning of next lap
                # or beginning of next lap less than 5 seconds away
                drv_data['PitOutTime'][lapcnt + 1] = to_timedelta(time)  # add to next lap
                pitstops += 1
            else:
                drv_data['PitOutTime'][lapcnt] = to_timedelta(time)  # add to current lap
                pitstops += 1

        # Get save information about personal best lap times at the timestamp
        # at which this information was received.
        # Whenever a lap is deleted (if that happens quickly after it was set),
        # the previous 'BestLapTime' value is sent again. There is some extra
        # logic at then end that correctly marks personal best laps based on
        # the data that is saved here.
        if val := recursive_dict_get(resp, 'BestLapTime', 'Value'):
            personal_best_lap_times.append(
                (to_timedelta(time), to_timedelta(val))
            )

        # Create approximate (sub)session (i.e. quali) split times by
        # (mis)using the session number counter from 'BestLapTimes'.
        # (Note: those lap times cannot be used for correct personal best
        #  detection, because the previous value is not resent here when a lap
        #  is deleted.)
        if (val := resp.get('BestLapTimes')) and isinstance(val, dict):
            session_n = int(list(val.keys())[0])
            if (session_n + 1) > len(session_split_times):
                session_split_times.append(to_timedelta(time))

        # new lap; create next row
        if 'NumberOfLaps' in resp and resp['NumberOfLaps'] > api_lapcnt:
            api_lapcnt += 1
            # make sure the car actually drove out of the pits already; it can't be a new lap if it didn't
            if pitstops >= 0:
                drv_data['Time'][lapcnt] = to_timedelta(time)
                drv_data['NumberOfLaps'][lapcnt] = lapcnt + 1  # don't use F1's lap count; ours is better
                drv_data['NumberOfPitStops'][lapcnt] = pitstops
                drv_data['Driver'][lapcnt] = drv
                lapcnt += 1
        
    if lapcnt == 0:  # no data at all for this driver
        return None, None

    # done reading the data, do postprocessing

    def data_in_lap(lap_n):
        relevant = ('Sector1Time', 'Sector2Time', 'Sector3Time', 'SpeedI1', 'SpeedI2',
                    'SpeedFL', 'SpeedST', 'LapTime')
        for col in relevant:
            if not pd.isnull(drv_data[col][lap_n]):
                return True
        return False

    # 'NumberOfLaps' always introduces a new lap (can be a previous one) but sometimes there is one more lap at the end
    # in this case the data will be added as usual above, lap count and pit stops are added here and the 'Time' is
    # calculated below from sector times
    if data_in_lap(lapcnt):
        drv_data['NumberOfLaps'][lapcnt] = lapcnt + 1
        drv_data['NumberOfPitStops'][lapcnt] = pitstops
        drv_data['Driver'][lapcnt] = drv
    else:  # if there was no more data after the last lap count increase,
        # delete the last empty record
        for key in drv_data.keys():
            drv_data[key] = drv_data[key][:-1]
    if not data_in_lap(0):  # remove first lap if there's no data;
        # "pseudo outlap" that didn't exist
        for key in drv_data.keys():
            drv_data[key] = drv_data[key][1:]
        drv_data['NumberOfLaps'] = list(map(lambda n: n - 1, drv_data['NumberOfLaps']))  # reduce each lap count by one

    if not drv_data['Time']:
        # ensure that there is still data left after potentially removing a lap
        return drv_data, session_split_times

    for i in range(len(drv_data['Time'])):
        sector_sum = datetime.timedelta(0)
        na_sectors = list()  # list of keys for missing sector times
        for key in ('Sector1Time', 'Sector2Time', 'Sector3Time'):
            st = drv_data[key][i]
            if pd.isna(st):
                na_sectors.append(key)
                continue
            sector_sum += st

        # check for incorrect lap times and remove them
        # fixes GH#167 among others
        if sector_sum > drv_data['LapTime'][i]:
            drv_data['LapTime'][i] = pd.NaT
            integrity_errors.append(i + 1)

        if i == 0:
            # only do following corrections for 2nd lap and onwards
            continue

        # The API only sends and update if a state changes, therefore, if two
        # lap times or sector times are exactly equal, the second value will
        # be missing. Missing sector times and lap times are calculated here
        # based on the available values for a lap (max one may be missing). If
        # the calculated value matches the previous value, it will be set.

        # lap time is missing
        if (not na_sectors) and pd.isna(drv_data['LapTime'][i]) \
                and (drv_data['LapTime'][i - 1] == sector_sum):

            drv_data['LapTime'][i] = sector_sum

        # one sector time is missing
        elif (len(na_sectors) == 1) and not pd.isna(drv_data['LapTime'][i]):
            # create a list with the two keys for available sector times
            ref_sec = ['Sector1Time', 'Sector2Time', 'Sector3Time']
            ref_sec.remove(na_sectors[0])

            if (sec1 := (drv_data['LapTime'][i]
                         - drv_data[ref_sec[0]][i]
                         - drv_data[ref_sec[1]][i])) \
                    == drv_data[na_sectors[0]][i - 1]:

                drv_data[na_sectors[0]][i] = sec1

    # lap time sync; check which sector time was triggered with the lowest latency
    # Sector3SessionTime == end of lap
    # Sector2SessionTime + Sector3Time == end of lap
    # Sector1SessionTime + Sector2Time + Sector3Time == end of lap
    # all of these three have slightly different times; take earliest one -> most exact because can't trigger too early
    for i in range(len(drv_data['Time'])):
        sector_sum = pd.Timedelta(0)
        min_time = drv_data['Time'][i]
        for sector_time, session_time in ((pd.Timedelta(0), drv_data['Sector3SessionTime'][i]),
                                          (drv_data['Sector3Time'][i], drv_data['Sector2SessionTime'][i]),
                                          (drv_data['Sector2Time'][i], drv_data['Sector1SessionTime'][i])):
            if pd.isnull(session_time):
                continue
            if pd.isnull(sector_time):
                break  # need to stop here because else the sector sum will be incorrect

            sector_sum += sector_time
            new_time = session_time + sector_sum
            if not pd.isnull(new_time) and (new_time < min_time or pd.isnull(min_time)):
                min_time = new_time
        if i > 0 and min_time < drv_data['Time'][i - 1]:
            integrity_errors.append(i + 1)  # not be possible if sector times and lap time are correct
            continue

        drv_data['Time'][i] = min_time

    # last lap needs to be removed if it does not have a 'Time' and it could not be calculated (likely an inlap)
    if pd.isnull(drv_data['Time'][-1]):
        if not pd.isnull(drv_data['PitInTime'][-1]):
            drv_data['Time'][-1] = drv_data['PitInTime'][-1]
        else:
            for key in drv_data.keys():
                drv_data[key] = drv_data[key][:-1]

    if not drv_data['Time']:
        # ensure that there is still data left after potentially removing a lap
        return drv_data, session_split_times

    # more lap sync, this time check which lap triggered with the lowest latency
    for i in range(len(drv_data['Time']) - 1, 0, -1):
        if (new_time := drv_data['Time'][i] - drv_data['LapTime'][i]) < \
                drv_data['Time'][i - 1]:
            if i > 1 and new_time < drv_data['Time'][i - 2]:
                integrity_errors.append(i + 1)  # not be possible if sector times and lap time are correct
            else:
                drv_data['Time'][i - 1] = new_time

    # need to go both directions once to make everything match up; also recalculate sector times
    for i in range(len(drv_data['Time']) - 1):
        if any(pd.isnull(tst) for tst in (
                drv_data['Time'][i], drv_data['LapTime'][i + 1],
                drv_data['Sector1Time'][i + 1],
                drv_data['Sector2Time'][i + 1],
                drv_data['Sector3Time'][i + 1])):
            continue  # lap not usable, missing critical values
        
        if (new_time := drv_data['Time'][i] + drv_data['LapTime'][i+1]) \
                < drv_data['Time'][i+1]:
            drv_data['Time'][i+1] = new_time
            # print(i+1, new_time)

        if (new_s1_time := drv_data['Time'][i]
                + drv_data['Sector1Time'][i+1]) \
                < drv_data['Sector1SessionTime'][i+1]:
            drv_data['Sector1SessionTime'][i+1] = new_s1_time
        if (new_s2_time := drv_data['Time'][i] + drv_data['Sector1Time'][i+1]
                + drv_data['Sector2Time'][i+1]) \
                < drv_data['Sector2SessionTime'][i+1]:
            drv_data['Sector2SessionTime'][i+1] = new_s2_time
        if (new_s3_time := drv_data['Time'][i] + drv_data['Sector1Time'][i+1]
                + drv_data['Sector2Time'][i+1]
                + drv_data['Sector3Time'][i+1]) \
                < drv_data['Sector3SessionTime'][i+1]:
            drv_data['Sector3SessionTime'][i+1] = new_s3_time

    # Iterate over list of personal lap times set 'IsPersonalBest'.
    # When a lap is deleted, the API resends the previous personal best.
    # Therefore, by iterating in reverse, if any lap is encountered that is
    # quicker than already processed personal best lap times, it must have
    # been deleted.
    # This is just best effort but not exhaustive as it can only handle lap
    # times that were deleted quickly (before the next personal best was set).
    _corrected_personal_best_lap_times = list()
    # list is only used for backreference within the loop
    cur_sn = len(session_split_times) - 1
    # current (sub)session number, personal best lap times need to be
    # considered for each (sub)session individually
    for time, pb_lap_time in reversed(personal_best_lap_times):
        if time < session_split_times[cur_sn]:
            # transitioned into the previous (sub)session (reverse iteration!)
            # reset the reference list, so time are considered individually
            cur_sn -= 1
            _corrected_personal_best_lap_times = list()

        if _corrected_personal_best_lap_times:
            if pb_lap_time in _corrected_personal_best_lap_times:
                continue
            elif pb_lap_time < min(_corrected_personal_best_lap_times):
                continue

        _corrected_personal_best_lap_times.append(pb_lap_time)

        # find the index of the corresponding lap by comparing with the lap
        # times and set 'IsPersonalBest' to True for that lap
        try:
            pb_idx = drv_data['LapTime'].index(pb_lap_time)
        except ValueError:
            # one example case where this error occurs, are wildly of personal
            # best times (>2 min lap time) that are sometimes present and
            # which have no corresponding lap time
            pass
        else:
            drv_data['IsPersonalBest'][pb_idx] = True

    # fix the number of pit stops; due to potentially multiple laps to the grid
    # where a car goes through the pit lane before finally taking its place
    # on the grid, the number of pit stops on the first lap may be already
    # greater than zero; therefore, apply correction so that we start with zero
    pitstop_offset = drv_data['NumberOfPitStops'][0]
    for i in range(len(drv_data['NumberOfPitStops'])):
        drv_data['NumberOfPitStops'][i] -= pitstop_offset

    # fix first lap PitInTime; same reason as above for pit stops, there may
    # be an incorrect PitInTime on the first lap. There always is a PitOutTime
    # for when the car leaves the box for the lap to the grid. There is a
    # PitInTime if the car drives multiple laps to the grid, discard these.
    # There is also a PitInTime if the car actually pits at the end of the
    # first lap, those need to be kept.
    if drv_data['PitInTime'][0] < drv_data['PitOutTime'][0]:
        drv_data['PitInTime'][0] = pd.NaT

    if integrity_errors:
        _logger.warning(
            f"Driver {drv: >2}: Encountered {len(integrity_errors)} timing "
            f"integrity error(s) near lap(s): {integrity_errors}.\n"
            f"This might be a bug and should be reported.")
    

    return drv_data


# for drv in resp_per_driver.keys():
#     drv_laps_data, drv_session_split_times \
#         = _laps_data_driver(resp_per_driver[drv], EMPTY_LAPS, drv)
#     print(drv)
#     print(drv_laps_data)
#     break

drv = "6"
drv_laps_data \
    = _laps_data_driver(resp_per_driver[drv], EMPTY_LAPS, drv)
pd.DataFrame(drv_laps_data)

,Time,Driver,LapTime,NumberOfLaps,NumberOfPitStops,PitOutTime,PitInTime,Sector1Time,Sector2Time,Sector3Time,Sector1SessionTime,Sector2SessionTime,Sector3SessionTime,SpeedI1,SpeedI2,SpeedFL,SpeedST,IsPersonalBest
0,0 days 01:04:03.247000,6,NaT,1,0,0 days 00:22:43.957000,NaT,NaT,0 days 00:00:40.348000,0 days 00:00:36.131000,NaT,0 days 01:03:27.185000,0 days 01:04:03.363000,279.0,277.0,212.0,302.0,False
1,0 days 01:05:37.792000,6,0 days 00:01:34.545000,2,0,NaT,NaT,0 days 00:00:19.131000,0 days 00:00:39.368000,0 days 00:00:36.046000,0 days 01:04:22.378000,0 days 01:05:01.746000,0 days 01:05:37.792000,286.0,303.0,207.0,298.0,True
2,0 days 01:07:11.028000,6,0 days 00:01:33.236000,3,0,NaT,NaT,0 days 00:00:18.869000,0 days 00:00:39.611000,0 days 00:00:34.756000,0 days 01:05:56.661000,0 days 01:06:36.272000,0 days 01:07:11.028000,284.0,297.0,211.0,317.0,True
3,0 days 01:08:43.332000,6,0 days 00:01:32.304000,4,0,NaT,NaT,0 days 00:00:18.654000,0 days 00:00:39.218000,0 days 00:00:34.432000,0 days 01:07:29.682000,0 days 01:08:08.900000,0 days 01:08:43.332000,279.0,296.0,210.0,304.0,True
4,0 days 01:10:15.087000,6,0 days 00:01:31.755000,5,0,NaT,NaT,0 days 00:00:18.719000,0 days 00:00:38.993000,0 days 00:00:34.043000,0 days 01:09:02.051000,0 days 01:09:41.044000,0 days 01:10:15.087000,284.0,305.0,209.0,307.0,True
5,0 days 01:11:47.090000,6,0 days 00:01:32.003000,6,0,NaT,NaT,0 days 00:00:18.689000,0 days 00:00:39.006000,0 days 00:00:34.308000,0 days 01:10:33.776000,0 days 01:11:12.782000,0 days 01:11:47.090000,279.0,302.0,211.0,304.0,False
6,0 days 01:13:18.468000,6,0 days 00:01:31.378000,7,0,NaT,NaT,0 days 00:00:18.538000,0 days 00:00:38.777000,0 days 00:00:34.063000,0 days 01:12:05.628000,0 days 01:12:44.405000,0 days 01:13:18.468000,284.0,301.0,211.0,307.0,True
7,0 days 01:14:50.263000,6,0 days 00:01:31.795000,8,0,NaT,NaT,0 days 00:00:18.591000,0 days 00:00:39.049000,0 days 00:00:34.155000,0 days 01:13:37.059000,0 days 01:14:16.108000,0 days 01:14:50.263000,286.0,300.0,210.0,302.0,False
8,0 days 01:16:21.967000,6,0 days 00:01:31.704000,9,0,NaT,NaT,0 days 00:00:18.538000,0 days 00:00:39.223000,0 days 00:00:33.943000,0 days 01:15:08.801000,0 days 01:15:48.024000,0 days 01:16:21.967000,277.0,289.0,212.0,299.0,False
9,0 days 01:17:53.356000,6,0 days 00:01:31.389000,10,0,NaT,NaT,0 days 00:00:18.554000,0 days 00:00:39.132000,0 days 00:00:33.703000,0 days 01:16:40.521000,0 days 01:17:19.653000,0 days 01:17:53.356000,279.0,295.0,211.0,298.0,False


In [28]:

pd.DataFrame(drv_laps_data)[pd.DataFrame(drv_laps_data).Time.isin([pd.to_timedelta(item[0]) for item in resp_per_driver[drv]])]

,Time,Driver,LapTime,NumberOfLaps,NumberOfPitStops,PitOutTime,PitInTime,Sector1Time,Sector2Time,Sector3Time,Sector1SessionTime,Sector2SessionTime,Sector3SessionTime,SpeedI1,SpeedI2,SpeedFL,SpeedST,IsPersonalBest
12,0 days 01:22:26.937000,6,0 days 00:01:31.078000,13,1,NaT,NaT,0 days 00:00:18.511000,0 days 00:00:38.886000,0 days 00:00:33.681000,0 days 01:21:14.387000,0 days 01:21:53.256000,0 days 01:22:27.037000,NaN,288.0,212.0,298.0,False
20,0 days 01:34:36.964000,6,0 days 00:01:30.886000,21,1,NaT,NaT,0 days 00:00:18.507000,0 days 00:00:38.758000,0 days 00:00:33.621000,0 days 01:33:24.590000,0 days 01:34:03.348000,0 days 01:34:36.964000,283.0,290.0,212.0,299.0,False
42,0 days 02:08:46.928000,6,0 days 00:01:29.337000,43,2,NaT,NaT,0 days 00:00:17.947000,0 days 00:00:38.332000,0 days 00:00:33.058000,0 days 02:07:35.538000,0 days 02:08:13.914000,0 days 02:08:47.027000,290.0,303.0,217.0,305.0,False


In [32]:
session.load()

core           INFO 	Loading data for Abu Dhabi Grand Prix - Race [v3.4.4]
req            INFO 	No cached data found for session_info. Loading data...
_api           INFO 	Fetching session info data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for driver_info. Loading data...
_api           INFO 	Fetching driver list...
req            INFO 	Data has been written to cache!
logger      WARNING 	Failed to load result data from Ergast!
core        WARNING 	No result data for this session available on Ergast! (This is expected for recent sessions)
req            INFO 	No cached data found for session_status_data. Loading data...
_api           INFO 	Fetching session status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for lap_count. Loading data...
_api           INFO 	Fetching lap count data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data

In [34]:
temp_laps = session.laps

In [38]:
temp_laps.pick_driver("6")

c:\Users\EU10698\venvs\goktugPy\Lib\site-packages\fastf1\core.py:3022: FutureWarning: pick_driver is deprecated and will be removed in a future release. Use pick_drivers instead.
  warnings.warn(("pick_driver is deprecated and will be removed"


,Time,Driver,DriverNumber,LapTime,LapNumber,Stint,PitOutTime,PitInTime,Sector1Time,Sector2Time,...,FreshTyre,Team,LapStartTime,LapStartDate,TrackStatus,Position,Deleted,DeletedReason,FastF1Generated,IsAccurate
806,0 days 01:04:03.241000,LAT,6,NaT,1.0,1.0,NaT,NaT,NaT,0 days 00:00:40.348000,...,True,Williams,0 days 01:02:21.871000,2021-12-12 13:03:21.881,1,16.0,False,,False,False
807,0 days 01:05:37.786000,LAT,6,0 days 00:01:34.545000,2.0,1.0,NaT,NaT,0 days 00:00:19.131000,0 days 00:00:39.368000,...,True,Williams,0 days 01:04:03.241000,2021-12-12 13:05:03.251,1,17.0,False,,False,True
808,0 days 01:07:11.022000,LAT,6,0 days 00:01:33.236000,3.0,1.0,NaT,NaT,0 days 00:00:18.869000,0 days 00:00:39.611000,...,True,Williams,0 days 01:05:37.786000,2021-12-12 13:06:37.796,1,17.0,False,,False,True
809,0 days 01:08:43.326000,LAT,6,0 days 00:01:32.304000,4.0,1.0,NaT,NaT,0 days 00:00:18.654000,0 days 00:00:39.218000,...,True,Williams,0 days 01:07:11.022000,2021-12-12 13:08:11.032,1,17.0,False,,False,True
810,0 days 01:10:15.081000,LAT,6,0 days 00:01:31.755000,5.0,1.0,NaT,NaT,0 days 00:00:18.719000,0 days 00:00:38.993000,...,True,Williams,0 days 01:08:43.326000,2021-12-12 13:09:43.336,1,17.0,False,,False,True
811,0 days 01:11:47.084000,LAT,6,0 days 00:01:32.003000,6.0,1.0,NaT,NaT,0 days 00:00:18.689000,0 days 00:00:39.006000,...,True,Williams,0 days 01:10:15.081000,2021-12-12 13:11:15.091,1,17.0,False,,False,True
812,0 days 01:13:18.462000,LAT,6,0 days 00:01:31.378000,7.0,1.0,NaT,NaT,0 days 00:00:18.538000,0 days 00:00:38.777000,...,True,Williams,0 days 01:11:47.084000,2021-12-12 13:12:47.094,1,17.0,False,,False,True
813,0 days 01:14:50.257000,LAT,6,0 days 00:01:31.795000,8.0,1.0,NaT,NaT,0 days 00:00:18.591000,0 days 00:00:39.049000,...,True,Williams,0 days 01:13:18.462000,2021-12-12 13:14:18.472,1,17.0,False,,False,True
814,0 days 01:16:21.961000,LAT,6,0 days 00:01:31.704000,9.0,1.0,NaT,NaT,0 days 00:00:18.538000,0 days 00:00:39.223000,...,True,Williams,0 days 01:14:50.257000,2021-12-12 13:15:50.267,1,17.0,False,,False,True
815,0 days 01:17:53.350000,LAT,6,0 days 00:01:31.389000,10.0,1.0,NaT,NaT,0 days 00:00:18.554000,0 days 00:00:39.132000,...,True,Williams,0 days 01:16:21.961000,2021-12-12 13:17:21.971,1,17.0,False,,False,True
